In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v3 as iio

def compute_cdf(image):
    """
    Compute the histogram and CDF of a grayscale image.
    Returns both the histogram and the CDF.
    """
    hist, _ = np.histogram(image.flatten(), bins=256, range=[0, 256])
    cdf = np.cumsum(hist).astype(np.float32)
    cdf /= cdf[-1]  # Normalize to [0,1]
    return hist, cdf

def ensure_numpy_array(cdf):
    """
    Ensure that the given CDF is a NumPy array.
    If it's a tuple, extract the first element and convert it to an array.
    """
    if isinstance(cdf, tuple):
        cdf = np.array(cdf[0])
    else:
        cdf = np.array(cdf)
    return cdf

def compute_inverse_cdf(cdf):
    """
    Compute the pseudo-inverse of a given CDF.
    Ensures smooth intensity mapping for histogram matching.
    """
    intensity_levels = np.linspace(0, 255, 256)  # 定义灰度级范围 0-255
    inverse_cdf = np.interp(np.linspace(0, 1, 256), cdf, intensity_levels)  # 修正插值范围
    return np.clip(np.round(inverse_cdf), 0, 255).astype(np.uint8)

def histogram_matching(source_image, target_image):
    """
    Perform histogram matching using corrected pixel mapping.
    """
    _, cdf_source = compute_cdf(source_image)  # Compute CDF of source image
    _, cdf_target = compute_cdf(target_image)  # Compute CDF of target image
    inverse_cdf_target = compute_inverse_cdf(cdf_target)  # Compute pseudo-inverse

    # Debugging: Check CDF values
    print("CDF Source Min/Max:", np.min(cdf_source), np.max(cdf_source))
    print("CDF Target Min/Max:", np.min(cdf_target), np.max(cdf_target))
    print("Inverse CDF Target Unique Values:", np.unique(inverse_cdf_target))

    # Step 1: Normalize source image intensity to CDF space
    source_cdf_values = cdf_source[source_image]

    # Step 2: Map source CDF values to target intensity values
    matched_image = np.interp(source_cdf_values, np.linspace(0, 1, 256), inverse_cdf_target)

    # Step 3: Convert to uint8 and clip values
    matched_image = np.clip(matched_image, 0, 255).astype(np.uint8)

    # Debugging: Check if image has reasonable intensity values
    print("Matched Image Min/Max:", np.min(matched_image), np.max(matched_image))
    print("Matched Image Unique Values:", np.unique(matched_image))

    return matched_image

# Load the source and target images
source_image = iio.imread("../TestImages/Week 1/AT3_1m4_01.tif")
target_image = iio.imread("../TestImages/Week 1/AT3_1m4_02.tif")

# Compute target CDF and inverse CDF
_, cdf_target = compute_cdf(target_image)
inverse_cdf_target = compute_inverse_cdf(cdf_target)

# Debugging: Check inverse CDF values
print("Inverse CDF Target:", np.unique(inverse_cdf_target))

# Perform histogram matching
matched_image = histogram_matching(source_image, target_image)

# Display the original, target, and matched images
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(source_image, cmap="gray")
plt.title("Source Image")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(target_image, cmap="gray")
plt.title("Target Image")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(matched_image, cmap="gray")
plt.title("Histogram Matched Image")
plt.axis("off")

plt.show()

# Compute CDFs for comparison
_, cdf_source = compute_cdf(source_image)
_, cdf_matched = compute_cdf(matched_image)

# Debugging: Ensure CDFs are correct
print("Matched Image Min/Max:", np.min(matched_image), np.max(matched_image))

# Plot CDF comparison
plt.figure(figsize=(8, 5))
plt.plot(range(256), cdf_source, label="Source CDF", color="red")
plt.plot(range(256), cdf_target, label="Target CDF", color="green")
plt.plot(range(256), cdf_matched, label="Matched CDF", color="blue")
plt.title("CDF Comparison")
plt.xlabel("Pixel Value")
plt.ylabel("Normalized CDF")
plt.legend()
plt.grid()
plt.show()
